In [ ]:
from VR_Trajectory_analysis import *
RUN_HIDDEN_CELLS = False  # Change to True when you need to run hidden cells

In [ ]:
if RUN_HIDDEN_CELLS:
    directory1 = '/Users/apaula/ownCloud/MatrexVR1/20250522_StaticDecisions/20250522_staticDecisionsV0_Data/RunData'
    directory2 = '/Users/apaula/ownCloud/MatrexVR3/20250522_VR3StaticDecisions_V0/20250522_VR3StaticDecisions_V0_Data/RunData'
    directory3 = '/Users/apaula/ownCloud/MatrexVR1/20250522_StaticDecisions/20250605_staticDecisions_V1_Data/RunData'
    directory4 = '/Users/apaula/ownCloud/MatrexVR3/20250522_VR3StaticDecisions_V0/20250605_VR3StaticDecisions_V1_Data/RunData'
    directory5 = '/Users/apaula/ownCloud/MatrexVR1/20250522_StaticDecisions/20250701_staticDecisions_V2_Data/RunData'
    directory6 = '/Users/apaula/ownCloud/MatrexVR3/20250522_VR3StaticDecisions_V0/20250701_VR3StaticDecisions_V2_Data/RunData'


In [ ]:
if RUN_HIDDEN_CELLS:
    df1 = get_combined_df(directory1, trim_seconds=1.0)
    df2 = get_combined_df(directory2, trim_seconds=1.0)
    df3 = get_combined_df(directory3, trim_seconds=1.0)
    df4 = get_combined_df(directory4, trim_seconds=1.0)
    df5 = get_combined_df(directory5, trim_seconds=1.0)
    df6 = get_combined_df(directory6, trim_seconds=1.0)
    concat_df = pd.concat([df1, df2, df3, df4, df5, df6], ignore_index=True)

In [ ]:
if RUN_HIDDEN_CELLS:
    concat_df = add_trial_id_and_displacement(concat_df)
    concat_df = add_trial_time(concat_df)

In [ ]:
if RUN_HIDDEN_CELLS:
    concat_df_path_pickle = '/Users/apaula/src/VRDataAnalysis/Ants/preprocessed_data/staticDecisions/staticDecisions_concat_df.pkl'
    concat_df.to_pickle(concat_df_path_pickle)

In [ ]:
concat_df_path_pickle = '/Users/apaula/src/VRDataAnalysis/Ants/preprocessed_data/staticDecisions/staticDecisions_concat_df.pkl'
concat_df = pd.read_pickle(concat_df_path_pickle)

In [ ]:
df_stationary, df_normal, df_excessive, stationary_ids, normal_ids, excessive_ids = classify_trials_by_path_length(concat_df[(concat_df['stepName'] != 'skybox') & (concat_df['TotalDisplacement'] > 12)], min_length=0, max_length=200)

In [ ]:
total_unique_trials = concat_df[(concat_df['stepName'] != 'skybox')]['UniqueTrialID'].nunique()

percent_stationary = df_stationary['UniqueTrialID'].nunique() / total_unique_trials * 100
percent_normal = df_normal['UniqueTrialID'].nunique() / total_unique_trials * 100
percent_excessive = df_excessive['UniqueTrialID'].nunique() / total_unique_trials * 100

percent_stationary, percent_normal, percent_excessive


In [ ]:
df_normal['UniqueTrialID'].nunique()*10/60/60/8

In [ ]:
concat_df['UniqueTrialID'].nunique()/df_normal['UniqueTrialID'].nunique()

In [ ]:
concat_df['UniqueTrialID'].nunique()*10/60/60/8

In [ ]:
#add angle column to df_joined
df_normal['Angle'] = df_normal['stepName'].apply(lambda x: parse_angle_from_config(x, default_angle=999.0))

In [ ]:
# -----------------------------------------------------------------
#  Trajectories for all  "BlueCylinder_BlueGreenCylinder" configs
#  shown in a grid, one panel per Angle
# -----------------------------------------------------------------
import math, re
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib import gridspec

# 1.  The joined dataframe is assumed to exist:  df_normal



df_filt = df_normal

# make sure Angle is numeric so the sort works
df_filt['Angle'] = pd.to_numeric(df_filt['Angle'], errors='coerce')

angles = sorted(df_filt['Angle'].dropna().unique())
if not angles:
    raise ValueError("No matching configs with an Angle column found.")

# 3.  Build grid layout -----------------------------------------------------
n_panels = len(angles)
n_cols   = 3                               # change to taste
n_rows   = math.ceil(n_panels / n_cols)

fig = plt.figure(figsize=(n_cols*4, n_rows*3.75))
gs  = gridspec.GridSpec(n_rows, n_cols, wspace=.25, hspace=.35)

# 4.  One sub-plot per Angle -------------------------------------------------
for idx, angle in enumerate(angles):
    ax = fig.add_subplot(gs[idx])
    g  = df_filt[df_filt['Angle'] == angle]

    # trajectories
    for trial_id, td in g.groupby('UniqueTrialID'):
        ax.plot(td['GameObjectPosX'], td['GameObjectPosZ'], alpha=.1)

    # basic aesthetics
    ax.set_xlim(-40, 40)
    ax.set_ylim(-10, 70)
    ax.set_aspect('equal', adjustable='box')

    ax.set_xticks([-60, -40, -20, 0, 20, 40, 60])
    ax.set_yticks([0, 20, 40, 60])
    ax.tick_params(labelsize=8)

    ax.set_title(f"{int(angle)}°", fontsize=10)


# hide any leftover empty cells
for j in range(idx+1, n_rows*n_cols):
    fig.add_subplot(gs[j]).axis('off')

fig.suptitle("Trajectories – BlueCylinder_BlueGreenCylinder", y=.92, fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
df_normal.columns

In [ ]:
# -------- 1. summarise to one row per trial -------------------------------
trial_summary = (
    df_normal
      .sort_values(['UniqueTrialID', 'elapsed_time'])
      .groupby('UniqueTrialID')
      .agg(
          FlyID=('FlyID', 'first'),  # or 'unique' if needed
          angle=('Angle', 'first'),
          travel_dir=('TravelDirectionDeg', 'first')
      )
      .dropna(subset=['angle', 'travel_dir'])
      .reset_index()
)

# convert Unity heading (0–360) → signed heading (-180…+180)  ⟹  0 in the middle
trial_summary['travel_dir_signed'] = (
    (trial_summary['travel_dir'] + 180) % 360    # shift, wrap into 0-360
    - 180                                        # back to signed range
)

# -------- 2. plot ----------------------------------------------------------
plt.figure(figsize=(12, 12))
plt.scatter(trial_summary['angle'],
            trial_summary['travel_dir_signed'],
            s=40, alpha=.1, color='black')

# -----------------------------------------------------------------
# NEW: goal-direction reference lines  (  y = ±½x  )
# -----------------------------------------------------------------
x_line = np.linspace(trial_summary['angle'].min() - 5,
                     trial_summary['angle'].max() + 5, 200)
plt.plot(x_line,  0.5 * x_line, ls='--', c='tab:red',  lw=0.5)
plt.plot(x_line, -0.5 * x_line, ls='--', c='tab:red', lw=0.5)

plt.xlabel('conflict angle (deg)')
plt.ylabel('Travel direction (deg, signed)')
plt.title('overall heading vs. conflict angle\n(one point per trial)')

plt.xlim(trial_summary['angle'].min() - 5,
         trial_summary['angle'].max() + 5)
plt.ylim(-180, 180)
plt.yticks(range(-180, 181, 60))
plt.axhline(0, color='grey', ls='--', lw=0.8)
plt.legend(frameon=False, fontsize=8)      # optional
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ---------- SETTINGS ----------

y_bin_centers = (y_bins[:-1] + y_bins[1:]) / 2

 # --- 3. Equal-count binning by angle using quantiles ---
n_xbins = 19
angle_values = trial_summary['angle'].values+ np.random.uniform(-0.01, 0.01, size=len(trial_summary))

quantile_edges = np.quantile(angle_values, np.linspace(0, 1, n_xbins + 1))

# Compute bin centers (for plotting)
x_bin_centers = 0.5 * (quantile_edges[:-1] + quantile_edges[1:])

trial_summary['angle_bin'] = pd.cut(
    trial_summary['angle'],
    bins=quantile_edges,
    labels=range(n_xbins),
    include_lowest=True,
    right=False  # or True, depending on whether right edge is inclusive
)

print(trial_summary['angle_bin'].isna().sum())  # should be 0


# --- 4. Histogram per bin ---
# Centered at 0

bin_width = 10
y_bins = np.arange(-180-bin_width/2, 180 + bin_width*3/2, bin_width)
y_bin_centers = (y_bins[:-1] + y_bins[1:]) / 2

heatmap = np.zeros((len(y_bin_centers), n_xbins))

for xbin in range(n_xbins):
    y_vals = trial_summary.loc[trial_summary['angle_bin'] == xbin, 'travel_dir_signed']
    counts, _ = np.histogram(y_vals, bins=y_bins)
    if counts.max() > 0:
        heatmap[:, xbin] = counts / counts.max()  # normalize each column


X, Y = np.meshgrid(quantile_edges, y_bins)

plt.figure(figsize=(10, 6))
plt.pcolormesh(X, Y, heatmap, shading='auto', cmap='viridis')

plt.colorbar(label='Normalized density (per angle bin)')
plt.xlabel('Conflict angle (deg)')
plt.ylabel('Travel direction (deg, signed)')
plt.title('Heatmap: Travel direction vs. conflict angle\n(equal-count angle bins, non-uniform spacing preserved)')

# Reference lines
x_line = np.linspace(quantile_edges[0], quantile_edges[-1], 500)
plt.plot(x_line,  0.5 * x_line, ls='--', c='red', lw=0.8)
plt.plot(x_line, -0.5 * x_line, ls='--', c='red', lw=0.8)
plt.axhline(0, color='grey', ls='--', lw=0.8)

plt.yticks(range(-180, 181, 60))
plt.tight_layout()
plt.show()




In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ---------- SETTINGS ----------

y_bin_centers = (y_bins[:-1] + y_bins[1:]) / 2

 # --- 3. Equal-count binning by angle using quantiles ---
n_xbins = 19
angle_values = trial_summary['angle'].values

quantile_edges = np.quantile(angle_values, np.linspace(0, 1, n_xbins + 1))

# Compute bin centers (for plotting)
x_bin_centers = 0.5 * (quantile_edges[:-1] + quantile_edges[1:])

trial_summary['angle_bin'] = pd.cut(
    trial_summary['angle'],
    bins=quantile_edges,
    labels=range(n_xbins),
    include_lowest=True,
    right=False  # or True, depending on whether right edge is inclusive
)

print(trial_summary['angle_bin'].isna().sum())  # should be 0


# --- 4. Histogram per bin ---
# Centered at 0

bin_width = 10
y_bins = np.arange(-180-bin_width/2, 180 + bin_width*3/2, bin_width)
y_bin_centers = (y_bins[:-1] + y_bins[1:]) / 2

heatmap = np.zeros((len(y_bin_centers), n_xbins))

for xbin in range(n_xbins):
    y_vals = trial_summary.loc[trial_summary['angle_bin'] == xbin, 'travel_dir_signed']
    counts, _ = np.histogram(y_vals, bins=y_bins)
    if counts.max() > 0:
        heatmap[:, xbin] = counts / counts.max()  # normalize each column


X, Y = np.meshgrid(quantile_edges, y_bins)

plt.figure(figsize=(10, 6))
plt.pcolormesh(X, Y, heatmap, shading='auto', cmap='viridis')

plt.colorbar(label='Normalized density (per angle bin)')
plt.xlabel('Conflict angle (deg)')
plt.ylabel('Travel direction (deg, signed)')
plt.title('Heatmap: Travel direction vs. conflict angle\n(equal-count angle bins, non-uniform spacing preserved)')

# Reference lines
x_line = np.linspace(quantile_edges[0], quantile_edges[-1], 500)
plt.plot(x_line,  0.5 * x_line, ls='--', c='red', lw=0.8)
plt.plot(x_line, -0.5 * x_line, ls='--', c='red', lw=0.8)
plt.axhline(0, color='grey', ls='--', lw=0.8)

plt.yticks(range(-180, 181, 60))
plt.tight_layout()
plt.show()




In [ ]:
# Mirror vertically across y = 0
df_mirrored = trial_summary.copy()
df_mirrored['travel_dir_signed'] = -df_mirrored['travel_dir_signed']

# Combine original + mirrored
df_augmented = pd.concat([trial_summary, df_mirrored], ignore_index=True)


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ---------- SETTINGS ----------

y_bin_centers = (y_bins[:-1] + y_bins[1:]) / 2

 # --- 3. Equal-count binning by angle using quantiles ---
n_xbins = 19
angle_values = df_augmented['angle'].values + np.random.uniform(-0.01, 0.01, size=len(df_augmented))

quantile_edges = np.quantile(angle_values, np.linspace(0, 1, n_xbins + 1))

# Compute bin centers (for plotting)
x_bin_centers = 0.5 * (quantile_edges[:-1] + quantile_edges[1:])

df_augmented['angle_bin'] = pd.cut(
    df_augmented['angle'],
    bins=quantile_edges,
    labels=range(n_xbins),
    include_lowest=True,
    right=False  # or True, depending on whether right edge is inclusive
)

print(df_augmented['angle_bin'].isna().sum())  # should be 0


# --- 4. Histogram per bin ---
# Centered at 0

bin_width = 5
y_bins = np.arange(-180-bin_width/2, 180 + bin_width*3/2, bin_width)
y_bin_centers = (y_bins[:-1] + y_bins[1:]) / 2

heatmap = np.zeros((len(y_bin_centers), n_xbins))

for xbin in range(n_xbins):
    y_vals = df_augmented.loc[df_augmented['angle_bin'] == xbin, 'travel_dir_signed']
    counts, _ = np.histogram(y_vals, bins=y_bins)
    if counts.max() > 0:
        heatmap[:, xbin] = counts / counts.max()  # normalize each column


X, Y = np.meshgrid(quantile_edges, y_bins)

plt.figure(figsize=(10, 6))
plt.pcolormesh(X, Y, heatmap, shading='auto', cmap='viridis')

plt.colorbar(label='Normalized density (per angle bin)')
plt.xlabel('Conflict angle (deg)')
plt.ylabel('Travel direction (deg, signed)')
plt.title('Heatmap: Travel direction vs. conflict angle\n(equal-count angle bins, non-uniform spacing preserved)')

# Reference lines
x_line = np.linspace(quantile_edges[0], quantile_edges[-1], 500)
plt.plot(x_line,  0.5 * x_line, ls='--', c='red', lw=0.8)
plt.plot(x_line, -0.5 * x_line, ls='--', c='red', lw=0.8)
plt.axhline(0, color='grey', ls='--', lw=0.8)

plt.yticks(range(-180, 181, 60))
plt.tight_layout()
plt.show()




In [ ]:
from scipy.ndimage import gaussian_filter


In [ ]:
# Apply Gaussian blur
heatmap_smooth = gaussian_filter(heatmap, sigma=[1.5, 1.5])  # [rows (y), cols (x)]


In [ ]:
plt.figure(figsize=(10, 8))
x_bin_centers = 0.5 * (quantile_edges[:-1] + quantile_edges[1:])
Xc, Yc = np.meshgrid(x_bin_centers, y_bin_centers)

plt.pcolormesh(X, Y, heatmap_smooth, shading='auto', cmap='viridis')
plt.contour(Xc, Yc, heatmap_smooth,
            levels=[0.3, 0.5, 0.7, 0.9],
            colors='white', linewidths=1.0)



#plt.colorbar(label='Normalized density (per angle bin, smoothed)')
plt.xlabel('Conflict angle (deg)')
plt.ylabel('Travel direction (deg, signed)')
plt.title('Smoothed heatmap: Travel direction vs. conflict angle')

# Reference lines
x_line = np.linspace(quantile_edges[0], quantile_edges[-1], 500)
plt.plot(x_line,  0.5 * x_line, ls='--', c='red', lw=0.8, alpha = 0.5)
plt.plot(x_line, -0.5 * x_line, ls='--', c='red', lw=0.8, alpha = 0.5
         )
#plt.axhline(0, color='grey', ls='--', lw=0.8)
plt.yticks(range(-180, 181, 60))
plt.tight_layout()
plt.show()


In [ ]:
# Preprocess: relative time per trial
df_normal['Relative Time'] = df_normal.groupby('UniqueTrialID')['Current Time'].transform(
    lambda x: (x - x.min()).dt.total_seconds()
)
# 3. Calculate step-wise time difference (in seconds)
df_normal['step_time'] = df_normal.groupby('UniqueTrialID')['Current Time'].diff().dt.total_seconds()

# 4. Calculate step-wise velocity
df_normal['step_velocity'] = df_normal['step_distance'] / df_normal['step_time']

# Replace infinite or NaN values with 0 (first row in each group will have NaN)
df_normal['step_velocity'] = df_normal['step_velocity'].replace([np.inf, -np.inf], np.nan).fillna(0)
df_normal['smoothed_velocity'] = df_normal['step_velocity'].rolling(window=7, center=True).mean()# Make a copy to avoid modifying original
df_analysis = df_normal.copy()

# Smooth the velocity per trial (7-step window, centered)
df_analysis['smoothed_velocity'] = (
    df_analysis.groupby('UniqueTrialID')['step_velocity']
    .transform(lambda x: x.rolling(window=7, center=True, min_periods=1).mean())
)

# Define relative time bins — you can tweak the bin size
bin_size = 0.25  # seconds
max_relative_time = df_analysis['Relative Time'].max()

bins = np.arange(0, max_relative_time + bin_size, bin_size)

# Assign each row to a time bin
df_analysis['time_bin'] = pd.cut(df_analysis['Relative Time'], bins=bins, right=False)

# Group by time bin and compute mean and SEM (optional) across all trials
summary = (
    df_analysis.groupby('time_bin')['smoothed_velocity']
    .agg(['mean', 'count', 'std'])
    .rename(columns={'mean': 'mean_velocity', 'std': 'std_velocity'})
)
summary['sem_velocity'] = summary['std_velocity'] / np.sqrt(summary['count'])

# Midpoint of each bin for plotting
summary['bin_center'] = [interval.left + bin_size / 2 for interval in summary.index]


In [ ]:
plt.figure(figsize=(12, 5))
plt.plot(summary['bin_center'], summary['mean_velocity'], label='Mean Velocity')
plt.fill_between(summary['bin_center'],
                 summary['mean_velocity'] - summary['sem_velocity'],
                 summary['mean_velocity'] + summary['sem_velocity'],
                 alpha=0.3, label='±1 SEM')
plt.xlabel('Relative Time (s)')
plt.ylabel('Mean Smoothed Velocity')
plt.title('Mean Smoothed Velocity Across All Trials Over Time')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Convert degrees to radians
theta_rad = np.deg2rad(df_analysis['GameObjectRotY'])

# Compute angle difference in radians (handles wrapping properly)
turn_rate_rad = np.angle(np.exp(1j * np.diff(theta_rad, prepend=np.nan)))

# Convert to degrees
df_analysis['turning_rate_deg'] = np.rad2deg(turn_rate_rad)

# Absolute turning rate (how much turning, regardless of direction)
df_analysis['abs_turning_rate'] = np.abs(df_analysis['turning_rate_deg'])

df_analysis['turning_rate_deg_per_sec'] = (
    df_analysis['turning_rate_deg'] / df_analysis['step_time']
).replace([np.inf, -np.inf], np.nan).fillna(0)

df_analysis['abs_turning_rate_per_sec'] = np.abs(df_analysis['turning_rate_deg_per_sec'])

# Compute mean turning rate per trial from the raw dataframe
trial_turning = (
    df_analysis
    .groupby('UniqueTrialID')['abs_turning_rate_per_sec']
    .mean()
    .reset_index()
    .rename(columns={'abs_turning_rate_per_sec': 'mean_turning_rate'})
)


# Get one Angle per trial
trial_angles = (
    df_analysis
    .groupby('UniqueTrialID')['Angle']
    .first()
    .reset_index()
)

# Merge it in
trial_turning = trial_turning.merge(trial_angles, on='UniqueTrialID')


angle_summary_raw = (
    trial_turning
    .groupby('Angle')['mean_turning_rate']
    .agg(['mean', 'std', 'count'])
    .reset_index()
)
angle_summary_raw['sem'] = angle_summary_raw['std'] / np.sqrt(angle_summary_raw['count'])

angle_summary_raw = (
    trial_turning
    .groupby('Angle')['mean_turning_rate']
    .agg(['mean', 'std', 'count'])
    .reset_index()
)
angle_summary_raw['sem'] = angle_summary_raw['std'] / np.sqrt(angle_summary_raw['count'])

plt.figure(figsize=(12, 5))
plt.errorbar(angle_summary_raw['Angle'], angle_summary_raw['mean'], yerr=angle_summary_raw['sem'],
             fmt='o', ecolor='gray', elinewidth=1, capsize=2, alpha=0.8)
plt.xlabel('Angle (degrees)')
plt.ylabel('Mean Turning Rate (°/s)')
plt.title('Mean Turning Rate by Raw Angle')
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# --- 1. Compute mean turning rate per trial ---
trial_turning = (
    df_analysis
    .groupby('UniqueTrialID')['abs_turning_rate_per_sec']
    .mean()
    .reset_index()
    .rename(columns={'abs_turning_rate_per_sec': 'mean_turning_rate'})
)

# --- 2. Merge trial-wise angles ---
trial_angles = (
    df_analysis
    .groupby('UniqueTrialID')['Angle']
    .first()
    .reset_index()
)

trial_turning = trial_turning.merge(trial_angles, on='UniqueTrialID')
trial_turning = trial_turning[trial_turning['mean_turning_rate'] <= 200]
# --- 3. Equal-count binning by Angle ---
n_bins = 20
angle_values = trial_turning['Angle'].values
quantile_edges = np.quantile(angle_values, np.linspace(0, 1, n_bins + 1))
bin_labels = 0.5 * (quantile_edges[:-1] + quantile_edges[1:])  # bin centers

# Bin each trial based on its angle
trial_turning['angle_bin'] = pd.cut(
    trial_turning['Angle'], bins=quantile_edges, labels=bin_labels, include_lowest=True
)

# Drop any NaNs from binning (shouldn’t happen, but safe)
binned_summary = (
    trial_turning
    .dropna(subset=['angle_bin'])
    .groupby('angle_bin')['mean_turning_rate']
    .agg(['mean', 'std', 'count'])
    .reset_index()
    .rename(columns={'angle_bin': 'Angle'})
)

# --- 4. Compute SEM ---
binned_summary['sem'] = binned_summary['std'] / np.sqrt(binned_summary['count'])

# --- 5. Plot ---
plt.figure(figsize=(12, 5))
plt.errorbar(binned_summary['Angle'].astype(float), binned_summary['mean'],
             yerr=binned_summary['sem'],
             fmt='o', ecolor='gray', elinewidth=1, capsize=2, alpha=0.8)

plt.xlabel('Angle (deg)')
plt.ylabel('Mean Turning Rate (°/s)')
plt.title('Mean Turning Rate (Equal-Count Bins)')
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
df_analysis.columns

In [ ]:
# --- 1. Compute mean smoothed velocity per trial ---
trial_velocity = (
    df_analysis
    .groupby('UniqueTrialID')['smoothed_velocity']
    .mean()
    .reset_index()
    .rename(columns={'smoothed_velocity': 'mean_smoothed_velocity'})
)

# --- 2. Merge trial-wise angles ---
trial_angles = (
    df_analysis
    .groupby('UniqueTrialID')['Angle']
    .first()
    .reset_index()
)

trial_velocity = trial_velocity.merge(trial_angles, on='UniqueTrialID')

# --- 3. Equal-count binning by Angle ---
n_bins = 20
angle_values = trial_velocity['Angle'].values
quantile_edges = np.quantile(angle_values, np.linspace(0, 1, n_bins + 1))
bin_labels = 0.5 * (quantile_edges[:-1] + quantile_edges[1:])  # bin centers

trial_velocity['angle_bin'] = pd.cut(
    trial_velocity['Angle'], bins=quantile_edges, labels=bin_labels, include_lowest=True
)

# --- 4. Aggregate by angle_bin ---
binned_summary = (
    trial_velocity
    .dropna(subset=['angle_bin'])
    .groupby('angle_bin')['mean_smoothed_velocity']
    .agg(['mean', 'std', 'count'])
    .reset_index()
    .rename(columns={'angle_bin': 'Angle'})
)

# --- 5. Compute SEM ---
binned_summary['sem'] = binned_summary['std'] / np.sqrt(binned_summary['count'])

# --- 6. Plot ---
plt.figure(figsize=(12, 5))
plt.errorbar(binned_summary['Angle'].astype(float), binned_summary['mean'],
             yerr=binned_summary['sem'],
             fmt='o', ecolor='gray', elinewidth=1, capsize=2, alpha=0.8)

plt.xlabel('Angle (deg)')
plt.ylabel('Mean Smoothed Velocity (units/s)')
plt.title('Mean Smoothed Velocity vs. Angle (Equal-Count Bins)')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

# --- 0. Prepare trial-level summary with FlyID, Angle, and mean turning rate ---
fly_ids = df_analysis.groupby('UniqueTrialID')['FlyID'].first().reset_index()

trial_turning = (
    df_analysis
    .groupby('UniqueTrialID')['abs_turning_rate_per_sec']
    .mean()
    .reset_index()
    .rename(columns={'abs_turning_rate_per_sec': 'mean_turning_rate'})
    .merge(
        df_analysis.groupby('UniqueTrialID')['Angle'].first().reset_index(),
        on='UniqueTrialID'
    )
    .merge(fly_ids, on='UniqueTrialID')
)

# --- 1. Remove outlier trials ---
trial_turning = trial_turning[trial_turning['mean_turning_rate'] <= 200]

# --- 2. Get top N flies by number of trials ---
n_flies = 9
top_flies = (
    trial_turning['FlyID']
    .value_counts()
    .nlargest(n_flies)
    .index
    .tolist()
)

# --- 3. Setup subplots ---
n_cols = 3
n_rows = math.ceil(n_flies / n_cols)
fig, axes = plt.subplots(n_rows, n_cols, figsize=(6 * n_cols, 5 * n_rows), sharey=True)
axes = axes.flatten()

n_bins = 20
for i, fly in enumerate(top_flies):
    fly_data = trial_turning[trial_turning['FlyID'] == fly]
    
    # Equal-count binning with duplicate edge removal
    angle_values = fly_data['Angle'].values
    raw_edges = np.quantile(angle_values, np.linspace(0, 1, n_bins + 1))
    quantile_edges = np.unique(raw_edges)  # remove duplicates

    if len(quantile_edges) - 1 < 5:
        print(f"⚠️  Skipping FlyID {fly}: too few unique angle bins ({len(quantile_edges) - 1})")
        continue

    bin_labels = 0.5 * (quantile_edges[:-1] + quantile_edges[1:])
    fly_data = fly_data.copy()  # avoid SettingWithCopyWarning
    fly_data['angle_bin'] = pd.cut(
        fly_data['Angle'],
        bins=quantile_edges,
        labels=bin_labels,
        include_lowest=True
    )
    
    # Group by bins and compute summary stats
    binned_summary = (
        fly_data
        .dropna(subset=['angle_bin'])
        .groupby('angle_bin')['mean_turning_rate']
        .agg(['mean', 'std', 'count'])
        .reset_index()
        .rename(columns={'angle_bin': 'Angle'})
    )
    binned_summary['sem'] = binned_summary['std'] / np.sqrt(binned_summary['count'])

    # --- 4. Plot ---
    ax = axes[i]
    ax.errorbar(binned_summary['Angle'].astype(float), binned_summary['mean'],
                yerr=binned_summary['sem'], fmt='o', capsize=2, ecolor='gray')
    ax.set_title(f'FlyID: {fly}')
    ax.set_xlabel('Angle (deg)')
    if i % n_cols == 0:
        ax.set_ylabel('Mean Turning Rate (°/s)')
    ax.grid(True)

# Remove any unused subplot axes
for j in range(len(top_flies), len(axes)):
    fig.delaxes(axes[j])

plt.suptitle(f'Mean Turning Rate by Angle — Top {n_flies} Flies', fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()


In [ ]:
# --- 5. Compute median + IQR using .agg() ---
binned_summary = (
    trial_turning
    .dropna(subset=['angle_bin'])
    .groupby('angle_bin')['mean_turning_rate']
    .agg(
        Median='median',
        Q1=lambda x: x.quantile(0.25),
        Q3=lambda x: x.quantile(0.75),
        Count='count'
    )
    .reset_index()
    .rename(columns={'angle_bin': 'Angle'})
)

# Compute IQR
binned_summary['IQR'] = binned_summary['Q3'] - binned_summary['Q1']

# --- 6. Plot median ± 0.5*IQR ---
plt.figure(figsize=(12, 5))
plt.errorbar(binned_summary['Angle'].astype(float), binned_summary['Median'],
             yerr=0.5 * binned_summary['IQR'],
             fmt='o', ecolor='gray', elinewidth=1, capsize=2, alpha=0.8)

plt.xlabel('Angle (deg)')
plt.ylabel('Median Turning Rate (°/s)')
plt.title('Median Turning Rate (Equal-Count Bins ± IQR/2)')
plt.grid(True)
plt.tight_layout()
plt.show()



In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Prepare data (absolute values)
x = np.abs(trial_summary['angle'].values)
y = np.abs(trial_summary['travel_dir_signed'].values)

# Error function for fixed model: 0 below x_c, 0.5*x above
def piecewise_error(x_c):
    y_pred = np.where(x < x_c, 0, 0.5 * x)
    return np.mean((y - y_pred) ** 2)

# Generate a range of critical angles to test
x_candidates = np.linspace(0, np.max(x), 500)
errors = np.array([piecewise_error(x_c) for x_c in x_candidates])

# Find best-fitting critical angle
best_idx = np.argmin(errors)
best_xc = x_candidates[best_idx]

print(f"Best critical angle: {best_xc:.2f}°")

# Plot the error curve
plt.figure(figsize=(8, 5))
plt.plot(x_candidates, errors, color='tab:blue')
plt.axvline(best_xc, color='tab:red', ls='--', label=f'Min error at {best_xc:.1f}°')
plt.xlabel('Candidate critical angle (deg)')
plt.ylabel('Mean Squared Error')
plt.title('Error vs. Critical Angle')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Prepare data (absolute values)
x = np.abs(trial_summary['angle'].values)
y = np.abs(trial_summary['travel_dir_signed'].values)

# Error function for fixed model: 0 below x_c, 0.5*x above
def piecewise_abs_error(x_c):
    y_pred = np.where(x < x_c, 0, 0.5 * x)
    return np.mean(np.abs(y - y_pred))  # MAE instead of MSE

# Generate a range of critical angles to test
x_candidates = np.linspace(0, np.max(x), 500)
errors = np.array([piecewise_abs_error(x_c) for x_c in x_candidates])

# Find best-fitting critical angle
best_idx = np.argmin(errors)
best_xc = x_candidates[best_idx]

print(f"Best critical angle (MAE): {best_xc:.2f}°")

# Plot the error curve
plt.figure(figsize=(8, 5))
plt.plot(x_candidates, errors, color='tab:blue')
plt.axvline(best_xc, color='tab:red', ls='--', label=f'Min error at {best_xc:.1f}°')
plt.xlabel('Candidate critical angle (deg)')
plt.ylabel('Mean Absolute Error')
plt.title('MAE vs. Critical Angle')
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Prepare data
x = np.abs(trial_summary['angle'].values)
y = np.abs(trial_summary['travel_dir_signed'].values)

# Fixed critical angle
fixed_xc = 75

# Model: 0 below threshold, 0.5 * x above
y_model = np.where(x < fixed_xc, 0, 0.5 * x)

# Plot data and model
plt.figure(figsize=(8, 6))
plt.scatter(x, y, alpha=0.1, color='black', label='Data')
plt.plot(np.sort(x), y_model[np.argsort(x)], color='tab:blue', label=f'Model (x_c = {fixed_xc}°)')
plt.axvline(fixed_xc, color='tab:red', ls='--', label='Critical angle = 75°')
plt.xlabel('ABS Conflict angle (deg)')
plt.ylabel('ABS Travel direction (deg)')
plt.title('Piecewise Model with Fixed Critical Angle = 75°')
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Prepare data
x = np.abs(trial_summary['angle'].values)
y = np.abs(trial_summary['travel_dir_signed'].values)

# Fixed critical angle
fixed_xc = 75

# Model: 0 below threshold, 0.5 * x above
y_model = np.where(x < fixed_xc, 0, 0.5 * x)

# Bin the data and compute median y per bin
bin_edges = np.linspace(0, max(x), 20)
bin_centers = 0.5 * (bin_edges[1:] + bin_edges[:-1])
median_y = [np.median(y[(x >= bin_edges[i]) & (x < bin_edges[i+1])]) for i in range(len(bin_edges)-1)]

# Plot
plt.figure(figsize=(8, 6))
plt.scatter(x, y, alpha=0.1, color='black', label='Data')
plt.plot(np.sort(x), y_model[np.argsort(x)], color='tab:blue', label=f'Model (x_c = {fixed_xc}°)')
plt.axvline(fixed_xc, color='tab:red', ls='--', label='Critical angle = 75°')
plt.plot(bin_centers, median_y, color='orange', marker='o', linestyle='-', label='Binned median')

plt.xlabel('ABS Conflict angle (deg)')
plt.ylabel('ABS Travel direction (deg)')
plt.title('Piecewise Model with Fixed Critical Angle = 75°')
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Prepare data
x = np.abs(trial_summary['angle'].values)
y = np.abs(trial_summary['travel_dir_signed'].values)

# Number of equal-count bins
n_bins = 40

# Compute quantile-based bin edges
quantile_edges = np.quantile(x, np.linspace(0, 1, n_bins + 1))
bin_centers = 0.5 * (quantile_edges[:-1] + quantile_edges[1:])

# For storing the mode (most frequent y range) per bin
y_mode = []

# Histogram bin settings for y (within each x-bin)
y_hist_bins =18

# Loop over quantile bins
for i in range(len(quantile_edges) - 1):
    mask = (x >= quantile_edges[i]) & (x < quantile_edges[i + 1])
    y_in_bin = y[mask]
    
    if len(y_in_bin) > 0:
        counts, y_edges = np.histogram(y_in_bin, bins=y_hist_bins)
        max_bin_index = np.argmax(counts)
        mode_center = 0.5 * (y_edges[max_bin_index] + y_edges[max_bin_index + 1])
        y_mode.append(mode_center)
    else:
        y_mode.append(np.nan)

# Plot
plt.figure(figsize=(8, 6))
plt.scatter(x, y, alpha=0.1, color='black', label='Data')
plt.plot(bin_centers, y_mode, color='purple', marker='o', linestyle='-', label='Histogram mode (equal-count bins)')

plt.xlabel('ABS Conflict angle (deg)')
plt.ylabel('ABS Travel direction (deg)')
plt.title('Histogram Mode Using Equal-Sample Bins')
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import make_interp_spline

# Prepare data
x = np.abs(trial_summary['angle'].values)
y = np.abs(trial_summary['travel_dir_signed'].values)

# Equal-count binning (quantiles)
n_bins = 40
quantile_edges = np.quantile(x, np.linspace(0, 1, n_bins + 1))
bin_centers = 0.5 * (quantile_edges[:-1] + quantile_edges[1:])

# Compute median y in each x-bin
median_y = []
for i in range(len(quantile_edges) - 1):
    mask = (x >= quantile_edges[i]) & (x < quantile_edges[i + 1])
    y_in_bin = y[mask]
    if len(y_in_bin) > 0:
        median_y.append(np.median(y_in_bin))
    else:
        median_y.append(np.nan)

# Clean NaNs
median_y = np.array(median_y)
valid = ~np.isnan(median_y)
x_fit = bin_centers[valid]
y_fit = median_y[valid]

# Fit B-spline to the median trend
spline = make_interp_spline(x_fit, y_fit, k=3)  # cubic spline
x_smooth = np.linspace(x_fit.min(), x_fit.max(), 100)
y_smooth = spline(x_smooth)

# Plot
plt.figure(figsize=(8, 6))
plt.scatter(x, y, alpha=0.1, color='black', label='Raw data')
plt.plot(bin_centers, median_y, 'o', color='orange', label='Binned median')


plt.xlabel('ABS Conflict angle (deg)')
plt.ylabel('ABS Travel direction (deg)')
plt.title('Smooth Median Trend with B-spline')
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# Define piecewise linear function
def piecewise_linear(x, x_c, a1, b1, a2, b2):
    return np.where(x < x_c, a1 * x + b1, a2 * x + b2)

# Fit model to binned median data
p0 = [x_fit.mean(), 0, 0, 1, 0]  # initial guess: xc, a1, b1, a2, b2
params, _ = curve_fit(piecewise_linear, x_fit, y_fit, p0=p0)
x_c, a1, b1, a2, b2 = params

# Evaluate model
x_model = np.linspace(x_fit.min(), x_fit.max(), 300)
y_model = piecewise_linear(x_model, *params)

# Plot
plt.figure(figsize=(8, 6))
plt.scatter(x, y, alpha=0.1, color='black', label='Raw data')
plt.plot(x_fit, y_fit, 'o', color='orange', label='Binned median')
plt.plot(x_model, y_model, color='green', label='Piecewise linear fit')
plt.axvline(x_c, color='red', linestyle='--', label=f'Transition at x = {x_c:.1f}')

plt.xlabel('ABS Conflict angle (deg)')
plt.ylabel('ABS Travel direction (deg)')
plt.title('Piecewise Linear Fit to Median Trend')
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# Piecewise linear with 3 segments and continuity
def piecewise_linear_3(x, x1, x2, a1, a2, a3, b1):
    b2 = (a1 - a2) * x1 + b1
    b3 = (a2 - a3) * x2 + b2
    return np.piecewise(
        x,
        [x < x1, (x >= x1) & (x < x2), x >= x2],
        [
            lambda x: a1 * x + b1,
            lambda x: a2 * x + b2,
            lambda x: a3 * x + b3
        ]
    )


In [ ]:
x_fit  # binned x (e.g. conflict angles)
y_fit  # binned medians of travel_dir_signed


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Count number of unique trials per FlyID
trials_per_fly = trial_summary.groupby('FlyID')['UniqueTrialID'].nunique().sort_values()

# Print a preview
print(trials_per_fly.head())

# Plot histogram
plt.figure(figsize=(8, 5))
plt.hist(trials_per_fly.values, bins=20, edgecolor='black')
plt.xlabel('Number of unique trials per Fly')
plt.ylabel('Number of flies')
plt.title('Distribution of Trial Counts per FlyID')
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# --- 1. Count trials per FlyID ---
fly_counts = trial_summary.groupby('FlyID')['UniqueTrialID'].nunique()
top_3_flies = fly_counts.sort_values(ascending=False).head(9).index.tolist()
print("Top 3 FlyIDs:", top_3_flies)

# --- 2. Define piecewise linear model with continuity ---
# Define model with fixed slopes a1 and a3
def piecewise_linear_fixed(x, x1, x2, a2, b1, a1_fixed=0.0, a3_fixed=0.5):
    b2 = (a1_fixed - a2) * x1 + b1
    b3 = (a2 - a3_fixed) * x2 + b2
    return np.piecewise(
        x,
        [x < x1, (x >= x1) & (x < x2), x >= x2],
        [
            lambda x: a1_fixed * x + b1,
            lambda x: a2 * x + b2,
            lambda x: a3_fixed * x + b3
        ]
    )


# --- 3. Fit and plot for each fly ---
for fly_id in top_3_flies:
    # Extract this fly's data
    df_fly = trial_summary[trial_summary['FlyID'] == fly_id]
    x = np.abs(df_fly['angle'].values)
    y = np.abs(df_fly['travel_dir_signed'].values)

    # Bin data and compute medians
    n_bins = 30
    quantile_edges = np.quantile(x, np.linspace(0, 1, n_bins + 1))
    x_bins = 0.5 * (quantile_edges[:-1] + quantile_edges[1:])
    y_medians = []

    for i in range(n_bins):
        mask = (x >= quantile_edges[i]) & (x < quantile_edges[i+1])
        if np.sum(mask) > 0:
            y_medians.append(np.median(y[mask]))
        else:
            y_medians.append(np.nan)

    x_fit = x_bins[~np.isnan(y_medians)]
    y_fit = np.array(y_medians)[~np.isnan(y_medians)]

    # Fit model with fixed a1 and a3
    p0 = [
        np.percentile(x_fit, 33),  # x1
        np.percentile(x_fit, 66),  # x2
        1.0,                       # a2 (middle slope)
        np.min(y_fit)              # b1
    ]

    try:
        params, _ = curve_fit(lambda x, x1, x2, a2, b1:
                            piecewise_linear_fixed(x, x1, x2, a2, b1, a1_fixed=0.0, a3_fixed=0.5),
                            x_fit, y_fit, p0=p0)

        # Evaluate model
        x_model = np.linspace(x_fit.min(), x_fit.max(), 300)
        y_model = piecewise_linear_fixed(x_model, *params)

        # Plot
        plt.figure(figsize=(8, 5))
        plt.scatter(x, y, alpha=0.1, color='gray', label='Raw data')
        plt.plot(x_fit, y_fit, 'o', color='orange', label='Binned medians')
        plt.plot(x_model, y_model, color='blue', label='Constrained piecewise fit')
        plt.axvline(params[0], color='red', linestyle='--', label=f'Transition 1: {params[0]:.1f}')
        plt.axvline(params[1], color='purple', linestyle='--', label=f'Transition 2: {params[1]:.1f}')

        plt.xlabel('ABS Conflict angle (deg)')
        plt.ylabel('ABS Travel direction (deg)')
        plt.title(f'Fly {fly_id} — Piecewise Fit (a1=0, a3=0.5)')
        plt.legend()
        plt.tight_layout()
        plt.show()

    except RuntimeError as e:
        print(f"Fit failed for Fly {fly_id}: {e}")



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# --- 1. Prepare full dataset ---
x = np.abs(trial_summary['angle'].values)
y = np.abs(trial_summary['travel_dir_signed'].values)

# Remove NaNs
valid = ~np.isnan(x) & ~np.isnan(y)
x = x[valid]
y = y[valid]

# --- 2. Bin and compute medians ---
n_bins = 20
quantile_edges = np.quantile(x, np.linspace(0, 1, n_bins + 1))
x_bins = 0.5 * (quantile_edges[:-1] + quantile_edges[1:])
y_medians = []

for i in range(n_bins):
    mask = (x >= quantile_edges[i]) & (x < quantile_edges[i+1])
    if np.sum(mask) > 0:
        y_medians.append(np.median(y[mask]))
    else:
        y_medians.append(np.nan)

x_fit = x_bins[~np.isnan(y_medians)]
y_fit = np.array(y_medians)[~np.isnan(y_medians)]

# --- 3. Define piecewise model with fixed slopes ---
def piecewise_linear_fixed(x, x1, x2, a2, b1, a1_fixed=0.0, a3_fixed=0.5):
    b2 = (a1_fixed - a2) * x1 + b1
    b3 = (a2 - a3_fixed) * x2 + b2
    return np.piecewise(
        x,
        [x < x1, (x >= x1) & (x < x2), x >= x2],
        [
            lambda x: a1_fixed * x + b1,
            lambda x: a2 * x + b2,
            lambda x: a3_fixed * x + b3
        ]
    )

# --- 4. Fit the model ---
p0 = [
    np.percentile(x_fit, 33),  # x1
    np.percentile(x_fit, 66),  # x2
    1.0,                       # a2 (middle slope)
    np.min(y_fit)              # b1 (baseline offset)
]

params, _ = curve_fit(lambda x, x1, x2, a2, b1:
                      piecewise_linear_fixed(x, x1, x2, a2, b1, a1_fixed=0.0, a3_fixed=0.5),
                      x_fit, y_fit, p0=p0)

# --- 5. Evaluate model ---
x_model = np.linspace(x_fit.min(), x_fit.max(), 300)
y_model = piecewise_linear_fixed(x_model, *params, a1_fixed=0.0, a3_fixed=0.5)

# --- 6. Plot ---
plt.figure(figsize=(10, 6))
plt.scatter(x, y, alpha=0.05, color='gray', label='Raw data')
plt.plot(x_fit, y_fit, 'o', color='orange', label='Binned medians')
plt.plot(x_model, y_model, color='blue', label='Constrained piecewise fit')
plt.axvline(params[0], color='red', linestyle='--', label=f'Transition 1: {params[0]:.1f}')
plt.axvline(params[1], color='purple', linestyle='--', label=f'Transition 2: {params[1]:.1f}')

plt.xlabel('ABS Conflict angle (deg)')
plt.ylabel('ABS Travel direction (deg)')
plt.title('Global 3-Segment Piecewise Linear Fit (a1=0, a3=0.5)')
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
# Sigmoid function
def sigmoid(x, L, x0, k, y0):
    return L / (1 + np.exp(-k * (x - x0))) + y0

# Initial guess: max, midpoint, steepness, y-offset
p0 = [np.max(y_fit) - np.min(y_fit), x_fit.mean(), 0.1, np.min(y_fit)]
params, _ = curve_fit(sigmoid, x_fit, y_fit, p0=p0)
L, x0, k, y0 = params

# Evaluate model
y_sigmoid = sigmoid(x_model, *params)

# Plot
plt.figure(figsize=(8, 6))
plt.scatter(x, y, alpha=0.1, color='black', label='Raw data')
plt.plot(x_fit, y_fit, 'o', color='orange', label='Binned median')
plt.plot(x_model, y_sigmoid, color='purple', label='Sigmoid fit')
plt.axvline(x0, color='red', linestyle='--', label=f'Center at x = {x0:.1f}')

plt.xlabel('ABS Conflict angle (deg)')
plt.ylabel('ABS Travel direction (deg)')
plt.title('Sigmoid Fit to Median Trend')
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# Sigmoid followed by linear ramp
def sigmoid_linear(x, L, x0, k, y0, slope):
    sigmoid = L / (1 + np.exp(-k * (x - x0))) + y0
    # Where sigmoid ends
    linear_start = x0 + 4 / k  # heuristic: sigmoid effectively ends ~4/k units after center
    # Create the hybrid function
    y = np.where(
        x <= linear_start,
        sigmoid,
        slope * (x - linear_start) + (L / (1 + np.exp(-k * (linear_start - x0))) + y0)
    )
    return y


In [ ]:
# Use binned median trend: x_fit, y_fit

# Initial guesses
L_init = max(y_fit) - min(y_fit)
x0_init = np.median(x_fit)
k_init = 0.1
y0_init = min(y_fit)
slope_init = 0.5

p0 = [L_init, x0_init, k_init, y0_init, slope_init]

# Fit
params, _ = curve_fit(sigmoid_linear, x_fit, y_fit, p0=p0)
L, x0, k, y0, slope = params

# Evaluate
x_model = np.linspace(min(x_fit), max(x_fit), 300)
y_model = sigmoid_linear(x_model, *params)

# Plot
plt.figure(figsize=(8, 6))
plt.scatter(x, y, alpha=0.1, color='black', label='Raw data')
plt.plot(x_fit, y_fit, 'o', color='orange', label='Binned median')
plt.plot(x_model, y_model, color='green', label='Sigmoid + linear fit')
plt.axvline(x0, color='red', linestyle='--', label=f'Sigmoid center: x = {x0:.1f}')

plt.xlabel('ABS Conflict angle (deg)')
plt.ylabel('ABS Travel direction (deg)')
plt.title('Sigmoid + Linear Fit to Median Trend')
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import LSQUnivariateSpline

# Prepare data
x_raw = np.abs(trial_summary['angle'].values)
y_raw = np.abs(trial_summary['travel_dir_signed'].values)

# Remove NaNs and sort
valid = ~np.isnan(x_raw) & ~np.isnan(y_raw)
x = x_raw[valid]
y = y_raw[valid]
sort_idx = np.argsort(x)
x = x[sort_idx]
y = y[sort_idx]

# Jitter duplicate x values slightly
_, idx, counts = np.unique(x, return_index=True, return_counts=True)
repeated_x = counts > 1
if np.any(repeated_x):
    for i in np.where(repeated_x)[0]:
        dup_indices = np.where(x == x[idx[i]])[0]
        x[dup_indices] += np.linspace(-1e-6, 1e-6, len(dup_indices))

# Try different knot counts
knot_counts = [4,8,16,32]  # You can adjust this list
x_smooth = np.linspace(x.min(), x.max(), 500)

plt.figure(figsize=(10, 6))
plt.scatter(x, y, alpha=0.05, color='gray', label='Raw data')

for n_knots in knot_counts:
    # Choose knots at quantiles
    knots = np.quantile(x, np.linspace(0, 1, n_knots + 2))[1:-1]  # drop first and last
    spline = LSQUnivariateSpline(x, y, t=knots, k=2)
    y_smooth = spline(x_smooth)
    plt.plot(x_smooth, y_smooth, label=f'{n_knots} knots')

plt.xlabel('ABS Conflict angle (deg)')
plt.ylabel('ABS Travel direction (deg)')
plt.title('B-spline Fit with Varying Knot Counts')
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import LSQUnivariateSpline

# Assume you already have x_sorted and y_sorted
# (cleaned, sorted, jittered)

# Fit spline with a few knots
knots = np.quantile(x_sorted, np.linspace(0, 1, 7))[1:-1]  # e.g., 3 internal knots
spline = LSQUnivariateSpline(x_sorted, y_sorted, t=knots, k=3)

# Define a dense evaluation grid
x_eval = np.linspace(x_sorted.min(), x_sorted.max(), 1000)
y_eval = spline(x_eval)

# Compute second derivative
y_second_deriv = spline.derivative(n=2)(x_eval)

# Find inflection points (zero crossings of second derivative)
sign_change = np.diff(np.sign(y_second_deriv))
inflection_indices = np.where(sign_change != 0)[0]
inflection_x = x_eval[inflection_indices]

# Plot
plt.figure(figsize=(10, 6))
plt.plot(x_sorted, y_sorted, '.', alpha=0.1, label='Raw data')
plt.plot(x_eval, y_eval, label='Spline fit', color='blue')
plt.axhline(0, color='gray', linestyle='--', alpha=0.4)
plt.plot(inflection_x, spline(inflection_x), 'ro', label='Inflection points')

plt.xlabel('ABS Conflict angle (deg)')
plt.ylabel('ABS Travel direction (deg)')
plt.title('Spline Fit with Inflection Points')
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
from scipy.interpolate import make_interp_spline
import numpy as np
import matplotlib.pyplot as plt

# Assume: bin_centers and y_mode already computed from previous step
# Remove NaNs for spline fitting
valid = ~np.isnan(y_mode)
x_fit = bin_centers[valid]
y_fit = np.array(y_mode)[valid]

# Fit B-spline (degree=3 is cubic spline)
spline = make_interp_spline(x_fit, y_fit, k=1)

# Create a smooth x-range for plotting
x_smooth = np.linspace(np.min(x_fit), np.max(x_fit), 300)
y_smooth = spline(x_smooth)

# Plot
plt.figure(figsize=(8, 6))
plt.scatter(x, y, alpha=0.1, color='black', label='Raw data')
plt.plot(bin_centers, y_mode, 'o', label='Histogram mode (binned)', color='gray')
plt.plot(x_smooth, y_smooth, label='B-spline fit', color='blue')

plt.xlabel('ABS Conflict angle (deg)')
plt.ylabel('ABS Travel direction (deg)')
plt.title('Smooth Mode Trend with B-spline')
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Prepare data
x = np.abs(trial_summary['angle'].values)
y = np.abs(trial_summary['travel_dir_signed'].values)

# Fixed critical angle
fixed_xc = 75

# Model: 0 below threshold, 0.5 * x above
y_model = np.where(x < fixed_xc, 0, 0.5 * x)

# Bin the data and compute median y per bin
bin_edges = np.linspace(0, max(x), 30)
bin_centers = 0.5 * (bin_edges[1:] + bin_edges[:-1])
median_y = [np.median(y[(x >= bin_edges[i]) & (x < bin_edges[i+1])]) for i in range(len(bin_edges)-1)]

# Plot
plt.figure(figsize=(8, 6))
plt.scatter(x, y, alpha=0.1, color='black', label='Data')
plt.plot(np.sort(x), y_model[np.argsort(x)], color='tab:blue', label=f'Model (x_c = {fixed_xc}°)')
plt.axvline(fixed_xc, color='tab:red', ls='--', label='Critical angle = 75°')
plt.plot(bin_centers, median_y, color='orange', marker='o', linestyle='-', label='Binned median')

plt.xlabel('ABS Conflict angle (deg)')
plt.ylabel('ABS Travel direction (deg)')
plt.title('Piecewise Model with Fixed Critical Angle = 75°')
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
trial_summary.head()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize

# Step 1: Prepare data
x = np.abs(trial_summary['angle'].values)
y = np.abs(trial_summary['travel_dir_signed'].values)

# Step 2: Define piecewise model and loss function
def model(x, a, x_c):
    return np.where(x < x_c, a, (1/2) * x)

def loss(params):
    a, x_c = params
    y_pred = model(x, a, x_c)
    return np.mean((y - y_pred) ** 2)

# Step 3: Fit parameters via optimization
initial_guess = [0, 60]
result = minimize(loss, initial_guess, bounds=[(-30, 30), (0, max(x))])

a_fit, x_c_fit = result.x
print(f"Fitted subcritical value (a): {a_fit:.2f}")
print(f"Fitted critical angle: {x_c_fit:.2f}°")

# Step 4: Plot data and model
x_fit = np.linspace(min(x), max(x), 300)
y_fit = model(x_fit, a_fit, x_c_fit)

plt.figure(figsize=(8, 6))
plt.scatter(x, y, alpha=0.1, label='Data', color='black')
plt.plot(x_fit, y_fit, color='tab:blue', label='Piecewise fit (a, x/3)')
plt.axvline(x_c_fit, color='tab:red', ls='--', label=f'Critical angle ≈ {x_c_fit:.1f}°')
plt.xlabel('ABS Conflict angle (deg)')
plt.ylabel('ABS Travel direction (deg)')
plt.title('Piecewise Fit: y = a (subcritical), y = x/3 (supercritical)')
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Step 1: Prepare data
x = np.abs(trial_summary['angle'].values)
y = np.abs(trial_summary['travel_dir_signed'].values)

# Step 2: Define the error function for fixed y=0 / slope=1/2 model
def piecewise_error(x_c):
    y_pred = np.where(x < x_c, 0, 0.5 * x)
    return np.mean((y - y_pred) ** 2)

# Step 3: Search for best critical angle
x_candidates = np.linspace(0, max(x), 500)
errors = np.array([piecewise_error(x_c) for x_c in x_candidates])
best_idx = np.argmin(errors)
best_xc = x_candidates[best_idx]

print(f"Best critical angle (fixed model): {best_xc:.2f}°")

# Step 4: Plot
x_fit = np.linspace(min(x), max(x), 300)
y_fit = np.where(x_fit < best_xc, 0, 0.5 * x_fit)

plt.figure(figsize=(8, 6))
plt.scatter(x, y, alpha=0.1, color='black', label='Data')
plt.plot(x_fit, y_fit, color='tab:blue', label='Model: y = 0 | x/2')
plt.axvline(best_xc, color='tab:red', ls='--', label=f'Critical angle ≈ {best_xc:.1f}°')
plt.xlabel('ABS Conflict angle (deg)')
plt.ylabel('ABS Travel direction (deg)')
plt.title('Piecewise Fit (Fixed y=0 Subcritical, Slope=1/2 Supercritical)')
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize

# Prepare data
x = np.abs(trial_summary['angle'].values)
y = np.abs(trial_summary['travel_dir_signed'].values)

# Piecewise model: constant 'a' below x_c, slope 'b' above
def piecewise_model(x, a, b, x_c):
    return np.where(x < x_c, a, b * x)

# Loss function: mean squared error between model and data
def loss(params):
    a, b, x_c = params
    y_pred = piecewise_model(x, a, b, x_c)
    return np.mean((y - y_pred) ** 2)

# Initial guess: [a, b, x_c]
initial_guess = [5, 0.5, 60]

# Bounds to keep things realistic
bounds = [(0, 30),      # a: subcritical baseline
          (0.1, 2),     # b: supercritical slope
          (0, max(x))]  # x_c: critical angle

# Fit the model
result = minimize(loss, x0=initial_guess, bounds=bounds)
a_fit, b_fit, xc_fit = result.x

print(f"Fitted parameters:")
print(f"  a (subcritical baseline): {a_fit:.2f}")
print(f"  b (supercritical slope):  {b_fit:.2f}")
print(f"  x_c (critical angle):     {xc_fit:.2f}°")

# Plot
x_fit = np.linspace(min(x), max(x), 300)
y_fit = piecewise_model(x_fit, a_fit, b_fit, xc_fit)

plt.figure(figsize=(8, 6))
plt.scatter(x, y, alpha=0.1, color='black', label='Data')
plt.plot(x_fit, y_fit, color='tab:blue', label='Piecewise fit')
plt.axvline(xc_fit, color='tab:red', ls='--', label=f'Critical angle ≈ {xc_fit:.1f}°')
plt.xlabel('ABS Conflict angle (deg)')
plt.ylabel('ABS Travel direction (deg)')
plt.title('Piecewise Fit: Flat below, Linear above Critical Angle')
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Ensure 'Current Time' is in datetime format
df_normal['Current Time'] = pd.to_datetime(df_normal['Current Time'])

# Preprocess: relative time per trial
df_normal['Relative Time'] = df_normal.groupby('UniqueTrialID')['Current Time'].transform(
    lambda x: (x - x.min()).dt.total_seconds()
)

# Sort just to be safe
df_normal = df_normal.sort_values(['Angle', 'UniqueTrialID', 'Relative Time'])

# Function to insert NaNs where centered angle jumps > threshold
def break_large_centered_jumps(group, threshold=300):
    # Convert to centered angles: [-180, 180)
    angle_centered = ((group['GameObjectRotY'] + 180) % 360) - 180
    diff = angle_centered.diff().abs()

    # Break jumps
    angle_broken = angle_centered.copy()
    angle_broken[diff > threshold] = np.nan

    group['AnglePlotCentered'] = angle_broken
    return group
df_normal = df_normal.groupby('UniqueTrialID', group_keys=False).apply(break_large_centered_jumps)

# Get all unique angles
unique_angles = sorted(df_normal['Angle'].dropna().unique())
n_angles = len(unique_angles)

# Create subplots
fig, axes = plt.subplots(n_angles, 1, figsize=(12, 4 * n_angles), sharex=False)

if n_angles == 1:
    axes = [axes]  # Ensure axes is iterable

for ax, angle in zip(axes, unique_angles):
    df_angle = df_normal[df_normal['Angle'] == angle]
    for trial_id in df_angle['UniqueTrialID'].unique():
        trial_data = df_angle[df_angle['UniqueTrialID'] == trial_id]
        ax.plot(trial_data['Relative Time'], trial_data['AnglePlotCentered'], label=f'Trial {trial_id}', alpha = 0.2)
    ax.set_xlim(0, 8)
    ax.set_title(f'Angle = {angle}')
    ax.set_ylabel('Travel Direction (Degrees)')
    #ax.legend(loc='upper right', bbox_to_anchor=(1.15, 1))

axes[-1].set_xlabel('Relative Time (seconds)')
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Ensure 'Current Time' is in datetime format
df_normal['Current Time'] = pd.to_datetime(df_normal['Current Time'])

# Preprocess: relative time per trial
df_normal['Relative Time'] = df_normal.groupby('UniqueTrialID')['Current Time'].transform(
    lambda x: (x - x.min()).dt.total_seconds()
)
# Compute initial orientation per trial (first GameObjectRotY)
initial_orientations = df_normal.sort_values('Relative Time').groupby('UniqueTrialID')['GameObjectRotY'].first()

# Normalize to [0, 1] for HSV color map (0-360 -> 0-1)
normalized_orientations = (initial_orientations % 360) / 360.0

# Map trial ID to color
import matplotlib.cm as cm
trial_colors = {trial_id: plt.cm.hsv(norm_val) for trial_id, norm_val in normalized_orientations.items()}

# Sort just to be safe
df_normal = df_normal.sort_values(['Angle', 'UniqueTrialID', 'Relative Time'])

# Function to insert NaNs where centered angle jumps > threshold
def break_large_centered_jumps(group, threshold=300):
    # Convert to centered angles: [-180, 180)
    angle_centered = ((group['GameObjectRotY'] + 180) % 360) - 180
    diff = angle_centered.diff().abs()

    # Break jumps
    angle_broken = angle_centered.copy()
    angle_broken[diff > threshold] = np.nan

    group['AnglePlotCentered'] = angle_broken
    return group
df_normal = df_normal.groupby('UniqueTrialID', group_keys=False).apply(break_large_centered_jumps)

# Get all unique angles
unique_angles = sorted(df_normal['Angle'].dropna().unique())
n_angles = len(unique_angles)

# Create subplots
fig, axes = plt.subplots(n_angles, 1, figsize=(12, 4 * n_angles), sharex=False)

if n_angles == 1:
    axes = [axes]  # Ensure axes is iterable

for ax, angle in zip(axes, unique_angles):
    df_angle = df_normal[df_normal['Angle'] == angle]
    for trial_id in df_angle['UniqueTrialID'].unique():
        trial_data = df_angle[df_angle['UniqueTrialID'] == trial_id]
        color = trial_colors.get(trial_id, 'gray')  # fallback if missing
        ax.plot(trial_data['Relative Time'], trial_data['AnglePlotCentered'],
                label=f'Trial {trial_id}', alpha=0.25, color=color)
    ax.set_xlim(0, 8)
    ax.set_title(f'Angle = {angle}')
    ax.set_ylabel('Travel Direction (Degrees)')


axes[-1].set_xlabel('Relative Time (seconds)')
plt.tight_layout()
plt.show()
